<div style="text-align:center; background-color:blue; padding:10px;">
  <h1 style="color:white;">EDA sur l'année 2022</h1>
</div>


<div style="text-align:left; background-color:gray; padding:10px;">
  <h1 style="color:white;">Import des librairies</h1>
</div>


In [1]:
# Visualisations et transorfmations de données
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# Librairies pour le scrapping
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Librairies pour la décompression de fichiers
import gzip
from io import BytesIO

<div style="text-align:left; background-color:gray; padding:10px;">
  <h1 style="color:white;">Scrapping de l'année 20022</h1>
</div>


In [2]:
# URL de la page web
url = 'https://files.data.gouv.fr/geo-dvf/latest/csv/'

response = requests.get(url)
html_content = response.content

# Utilisez BeautifulSoup pour analyser le HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Trouver toutes les balises <a>
a_tags = soup.find_all('a')
annees=[]
# Parcourir chaque balise <a> et extraire le texte ainsi que la date/heure
for a_tag in a_tags:
    name = a_tag.text.strip()
    if name!="../":
        annees.append(name)
print(annees)

['2018/', '2019/', '2020/', '2021/', '2022/', '2023/']


In [3]:
# Récupération de l'avant denière année pour avoir une année comlpéte
url = f'https://files.data.gouv.fr/geo-dvf/latest/csv/{annees[-2]}'
response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')

# Trouver la balise 'a' qui contient les liens de téléchargement
csv_element = soup.find('pre').find('a', {'href': "full.csv.gz"})
csv_link = urljoin(url, csv_element['href'])
print(csv_link)

https://files.data.gouv.fr/geo-dvf/latest/csv/2022/full.csv.gz


In [4]:
csv_response = requests.get(csv_link)
if csv_response.status_code == 200:
    # Utilisation du buffer pour décompresser le fichier .gz
    with BytesIO(csv_response.content) as file_buffer:
        with gzip.GzipFile(fileobj=file_buffer, mode='rb') as gz_file:
            df = pd.read_csv(gz_file, low_memory=False)
            df = df.drop_duplicates()
else :
    print("Erreur !!!")

<div style="text-align:left; background-color:gray; padding:10px;">
  <h1 style="color:white;">EDA</h1>
</div>


In [5]:
df.head()

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,ancien_code_commune,ancien_nom_commune,id_parcelle,ancien_id_parcelle,numero_volume,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
0,2022-1,2022-01-03,1,Vente,55000.0,13.0,NaN,RUE DE LA LIBERTE,2280,1000.0,01053,Bourg-en-Bresse,01,NaN,NaN,01053000AM0102,NaN,NaN,7,24.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0,Appartement,24.0,1.0,NaN,NaN,NaN,NaN,NaN,5.218712,46.198053
1,2022-2,2022-01-03,1,Vente,143000.0,NaN,NaN,CHAMP COCHET,B010,1480.0,01398,Savigneux,01,NaN,NaN,01398000ZE0184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,S,sols,NaN,NaN,84.0,4.847863,46.000587
2,2022-2,2022-01-03,1,Vente,143000.0,NaN,NaN,CHAMP COCHET,B010,1480.0,01398,Savigneux,01,NaN,NaN,01398000ZE0185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,S,sols,NaN,NaN,88.0,4.847924,46.000581
3,2022-2,2022-01-03,1,Vente,143000.0,98.0,NaN,RTE DE LA DOMBES,0055,1480.0,01398,Savigneux,01,NaN,NaN,01398000ZE0187,NaN,NaN,1,123.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0,Appartement,140.0,3.0,NaN,NaN,NaN,NaN,NaN,4.848340,46.000631
4,2022-3,2022-01-04,1,Vente,300.0,NaN,NaN,AUX PIERRES,B031,1480.0,01243,Messimy-sur-Saône,01,NaN,NaN,012430000A0643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,T,terres,NaN,NaN,510.0,4.747594,46.045423


### Sélection des variables demandées dans l'expression des besoins


In [6]:
df = df.loc[df.nature_mutation=="Vente",
                                        ["id_mutation",
                                            "date_mutation","valeur_fonciere",
                                           "adresse_numero","adresse_nom_voie",
                                           "code_postal","code_commune",
                                           "type_local","nombre_pieces_principales",
                                           "surface_reelle_bati","surface_terrain",
                                           "longitude", "latitude"]]
df.head(20)

,id_mutation,date_mutation,valeur_fonciere,adresse_numero,adresse_nom_voie,code_postal,code_commune,type_local,nombre_pieces_principales,surface_reelle_bati,surface_terrain,longitude,latitude
0,2022-1,2022-01-03,55000.0,13.0,RUE DE LA LIBERTE,1000.0,01053,Appartement,1.0,24.0,NaN,5.218712,46.198053
1,2022-2,2022-01-03,143000.0,NaN,CHAMP COCHET,1480.0,01398,NaN,NaN,NaN,84.0,4.847863,46.000587
2,2022-2,2022-01-03,143000.0,NaN,CHAMP COCHET,1480.0,01398,NaN,NaN,NaN,88.0,4.847924,46.000581
3,2022-2,2022-01-03,143000.0,98.0,RTE DE LA DOMBES,1480.0,01398,Appartement,3.0,140.0,NaN,4.848340,46.000631
4,2022-3,2022-01-04,300.0,NaN,AUX PIERRES,1480.0,01243,NaN,NaN,NaN,510.0,4.747594,46.045423
5,2022-4,2022-01-06,255000.0,282.0,RTE DE POISATON,1560.0,01230,Maison,5.0,108.0,649.0,5.103407,46.422348
6,2022-4,2022-01-06,255000.0,NaN,LA FREGONIERE,1560.0,01230,NaN,NaN,NaN,360.0,5.103355,46.422257
7,2022-5,2022-01-03,525000.0,NaN,PL DE LA CROIX BLANCHE,1390.0,01333,NaN,NaN,NaN,50.0,4.951399,45.926420
8,2022-5,2022-01-03,525000.0,217.0,PL DE LA CROIX BLANCHE,1390.0,01333,Appartement,4.0,126.0,628.0,4.951266,45.926511
9,2022-5,2022-01-03,525000.0,217.0,PL DE LA CROIX BLANCHE,1390.0,01333,Local industriel. commercial ou assimilé,0.0,424.0,628.0,4.951266,45.926511


### On peut voir dans le dataframe ci dessus qu'une vente(Id_mutation) peut avoir plusieures lignes  


In [7]:
print("Nombre de lignes dans le dataframe de l'année 2022 :",df.shape[0])
print("Nombre de vente en 2022 : ", df.id_mutation.nunique())

Nombre de lignes dans le dataframe de l'année 2022 : 3926112
Nombre de vente en 2022 :  1528975


### Nous voulons récupérer toutes les ventes qui concerne un appartement ou une maison


In [8]:

# Récupération de tous les id_mutation comportant Appartement ou Maison
valid_ids = df[df['type_local'].isin(['Appartement', 'Maison'])]['id_mutation'].unique()

# Filtrage pour récupérer toutes les lignes pour les ventes qui concerne un appartement ou une maison
df = df[df['id_mutation'].isin(valid_ids)]

In [9]:
df.head(20)

,id_mutation,date_mutation,valeur_fonciere,adresse_numero,adresse_nom_voie,code_postal,code_commune,type_local,nombre_pieces_principales,surface_reelle_bati,surface_terrain,longitude,latitude
0,2022-1,2022-01-03,55000.0,13.0,RUE DE LA LIBERTE,1000.0,01053,Appartement,1.0,24.0,NaN,5.218712,46.198053
1,2022-2,2022-01-03,143000.0,NaN,CHAMP COCHET,1480.0,01398,NaN,NaN,NaN,84.0,4.847863,46.000587
2,2022-2,2022-01-03,143000.0,NaN,CHAMP COCHET,1480.0,01398,NaN,NaN,NaN,88.0,4.847924,46.000581
3,2022-2,2022-01-03,143000.0,98.0,RTE DE LA DOMBES,1480.0,01398,Appartement,3.0,140.0,NaN,4.848340,46.000631
5,2022-4,2022-01-06,255000.0,282.0,RTE DE POISATON,1560.0,01230,Maison,5.0,108.0,649.0,5.103407,46.422348
6,2022-4,2022-01-06,255000.0,NaN,LA FREGONIERE,1560.0,01230,NaN,NaN,NaN,360.0,5.103355,46.422257
7,2022-5,2022-01-03,525000.0,NaN,PL DE LA CROIX BLANCHE,1390.0,01333,NaN,NaN,NaN,50.0,4.951399,45.926420
8,2022-5,2022-01-03,525000.0,217.0,PL DE LA CROIX BLANCHE,1390.0,01333,Appartement,4.0,126.0,628.0,4.951266,45.926511
9,2022-5,2022-01-03,525000.0,217.0,PL DE LA CROIX BLANCHE,1390.0,01333,Local industriel. commercial ou assimilé,0.0,424.0,628.0,4.951266,45.926511
11,2022-7,2022-01-05,64000.0,12.0,BD DE BROU,1000.0,01053,Dépendance,0.0,NaN,NaN,5.229155,46.204984


In [10]:
print("Type de local :\n", df.type_local.value_counts())

Type de local :
 type_local
Dépendance                                  836330
Maison                                      746048
Appartement                                 574206
Local industriel. commercial ou assimilé     43716
Name: count, dtype: int64


### L'id_mutation 2022-5 posséde 2 types de local : Appartement et Local industriel commercial ou assimilé. Alors qu'une dépendances peut faire partie d'une maison. Il faut donc supprimer tous les id_mutation comportant un localindustiel

In [11]:
# Récupération de tous les id_mutation comportant Appartement ou Maison
not_valid_ids = df[df['type_local'].isin(['Local industriel. commercial ou assimilé'])]['id_mutation'].unique()

df = df[~df['id_mutation'].isin(not_valid_ids)]
print("Type de local :\n", df.type_local.value_counts())

Type de local :
 type_local
Dépendance     814454
Maison         722206
Appartement    531056
Name: count, dtype: int64


### à quel type de bien est rattaché une dépendances

In [12]:
# Recherche d'id_mutation comportant une dépendance
id_with_dependance= df[df['type_local'].isin(['Dépendance'])]['id_mutation'].unique()

# Ajout d'une colonne dépendance et 
df['dependance'] = df['id_mutation'].isin(id_with_dependance).astype(int)

# Remplacement de Dépendance dans type_local par Nan
df['type_local'] = df['type_local'].replace('Dépendance', np.nan)

df.head(10)

,id_mutation,date_mutation,valeur_fonciere,adresse_numero,adresse_nom_voie,code_postal,code_commune,type_local,nombre_pieces_principales,surface_reelle_bati,surface_terrain,longitude,latitude,dependance
0,2022-1,2022-01-03,55000.0,13.0,RUE DE LA LIBERTE,1000.0,01053,Appartement,1.0,24.0,NaN,5.218712,46.198053,0
1,2022-2,2022-01-03,143000.0,NaN,CHAMP COCHET,1480.0,01398,NaN,NaN,NaN,84.0,4.847863,46.000587,0
2,2022-2,2022-01-03,143000.0,NaN,CHAMP COCHET,1480.0,01398,NaN,NaN,NaN,88.0,4.847924,46.000581,0
3,2022-2,2022-01-03,143000.0,98.0,RTE DE LA DOMBES,1480.0,01398,Appartement,3.0,140.0,NaN,4.848340,46.000631,0
5,2022-4,2022-01-06,255000.0,282.0,RTE DE POISATON,1560.0,01230,Maison,5.0,108.0,649.0,5.103407,46.422348,0
6,2022-4,2022-01-06,255000.0,NaN,LA FREGONIERE,1560.0,01230,NaN,NaN,NaN,360.0,5.103355,46.422257,0
11,2022-7,2022-01-05,64000.0,12.0,BD DE BROU,1000.0,01053,NaN,0.0,NaN,NaN,5.229155,46.204984,1
12,2022-7,2022-01-05,64000.0,12.0,BD DE BROU,1000.0,01053,Appartement,2.0,117.0,NaN,5.229155,46.204984,1
13,2022-8,2022-01-03,140000.0,6.0,RUE DE BEAUGENCY,1000.0,01053,Maison,4.0,100.0,796.0,5.250556,46.209586,0
14,2022-8,2022-01-03,140000.0,NaN,RUE DE BEAUGENCY,1000.0,01053,NaN,NaN,NaN,260.0,5.249823,46.209593,0


### Un id_mutation peut il avoir un appartement et une maison ? 
### Si une vente comporte une maison et un appartement alors ils seront supprimés car il est impossible de distinguer le prix pour la partie de la maison et celle de l'appartment.

In [13]:
df_filtered = df.groupby('id_mutation').filter(lambda x: ('Appartement' in x['type_local'].values) and ('Maison' in x['type_local'].values))
df_filtered.head()

,id_mutation,date_mutation,valeur_fonciere,adresse_numero,adresse_nom_voie,code_postal,code_commune,type_local,nombre_pieces_principales,surface_reelle_bati,surface_terrain,longitude,latitude,dependance
256,2022-99,2022-01-07,230000.0,3.0,RUE DE LA RESISTANCE,1500.0,01004,Appartement,1.0,26.0,402.0,5.347792,45.957224,1
257,2022-99,2022-01-07,230000.0,3.0,RUE DE LA RESISTANCE,1500.0,01004,Appartement,1.0,27.0,402.0,5.347792,45.957224,1
258,2022-99,2022-01-07,230000.0,3.0,RUE DE LA RESISTANCE,1500.0,01004,Maison,5.0,131.0,402.0,5.347792,45.957224,1
259,2022-99,2022-01-07,230000.0,3.0,RUE DE LA RESISTANCE,1500.0,01004,NaN,0.0,NaN,402.0,5.347792,45.957224,1
260,2022-99,2022-01-07,230000.0,3.0,RUE DE LA RESISTANCE,1500.0,01004,Appartement,2.0,42.0,402.0,5.347792,45.957224,1


In [14]:
# Liste des id_mutation
liste_bien_double = df_filtered.id_mutation.unique()

# Retrait des lignes
df = df[~df['id_mutation'].isin(liste_bien_double)]

# Vérification
df_filtered2 = df.groupby('id_mutation').filter(lambda x: ('Appartement' in x['type_local'].values) and ('Maison' in x['type_local'].values))
df_filtered2.head()

,id_mutation,date_mutation,valeur_fonciere,adresse_numero,adresse_nom_voie,code_postal,code_commune,type_local,nombre_pieces_principales,surface_reelle_bati,surface_terrain,longitude,latitude,dependance


In [15]:
df.head(10)

,id_mutation,date_mutation,valeur_fonciere,adresse_numero,adresse_nom_voie,code_postal,code_commune,type_local,nombre_pieces_principales,surface_reelle_bati,surface_terrain,longitude,latitude,dependance
0,2022-1,2022-01-03,55000.0,13.0,RUE DE LA LIBERTE,1000.0,01053,Appartement,1.0,24.0,NaN,5.218712,46.198053,0
1,2022-2,2022-01-03,143000.0,NaN,CHAMP COCHET,1480.0,01398,NaN,NaN,NaN,84.0,4.847863,46.000587,0
2,2022-2,2022-01-03,143000.0,NaN,CHAMP COCHET,1480.0,01398,NaN,NaN,NaN,88.0,4.847924,46.000581,0
3,2022-2,2022-01-03,143000.0,98.0,RTE DE LA DOMBES,1480.0,01398,Appartement,3.0,140.0,NaN,4.848340,46.000631,0
5,2022-4,2022-01-06,255000.0,282.0,RTE DE POISATON,1560.0,01230,Maison,5.0,108.0,649.0,5.103407,46.422348,0
6,2022-4,2022-01-06,255000.0,NaN,LA FREGONIERE,1560.0,01230,NaN,NaN,NaN,360.0,5.103355,46.422257,0
11,2022-7,2022-01-05,64000.0,12.0,BD DE BROU,1000.0,01053,NaN,0.0,NaN,NaN,5.229155,46.204984,1
12,2022-7,2022-01-05,64000.0,12.0,BD DE BROU,1000.0,01053,Appartement,2.0,117.0,NaN,5.229155,46.204984,1
13,2022-8,2022-01-03,140000.0,6.0,RUE DE BEAUGENCY,1000.0,01053,Maison,4.0,100.0,796.0,5.250556,46.209586,0
14,2022-8,2022-01-03,140000.0,NaN,RUE DE BEAUGENCY,1000.0,01053,NaN,NaN,NaN,260.0,5.249823,46.209593,0


### Avant de grouper les données sur les id_mutation il faut formater les données et gérer les Nan

In [16]:
# Visualisation des nan
df.isna().sum()

id_mutation                        0
date_mutation                      0
valeur_fonciere                 3110
adresse_numero                485256
adresse_nom_voie                 191
code_postal                      196
code_commune                       0
type_local                   1335015
nombre_pieces_principales     524147
surface_reelle_bati          1335084
surface_terrain              1026932
longitude                      16309
latitude                       16309
dependance                         0
dtype: int64

In [17]:
# Visualisation des Nan dans valeur fonciére
liste_id_with_nan_valeur_fonciere = df.loc[pd.isna(df['valeur_fonciere']), 'id_mutation'].unique()
df_filtered = df[df['id_mutation'].isin(liste_id_with_nan_valeur_fonciere)]
df_filtered.head(10)

,id_mutation,date_mutation,valeur_fonciere,adresse_numero,adresse_nom_voie,code_postal,code_commune,type_local,nombre_pieces_principales,surface_reelle_bati,surface_terrain,longitude,latitude,dependance
40487,2022-13988,2022-11-07,NaN,NaN,CHAMPIONNIERE,1500.0,01007,NaN,NaN,NaN,23.0,5.325069,45.999380,1
40488,2022-13988,2022-11-07,NaN,33.0,IMP DU VERGER,1500.0,01007,NaN,0.0,NaN,150.0,5.324994,45.999384,1
40489,2022-13988,2022-11-07,NaN,33.0,IMP DU VERGER,1500.0,01007,Maison,3.0,79.0,150.0,5.324994,45.999384,1
40490,2022-13988,2022-11-07,NaN,NaN,LE VERGER,1500.0,01007,NaN,NaN,NaN,2380.0,5.325812,45.998957,1
40491,2022-13988,2022-11-07,NaN,NaN,LE VERGER,1500.0,01007,NaN,NaN,NaN,380.0,5.326384,45.998718,1
40492,2022-13988,2022-11-07,NaN,25.0,IMP DU VERGER,1500.0,01007,NaN,0.0,NaN,467.0,5.325067,45.999509,1
40493,2022-13988,2022-11-07,NaN,25.0,IMP DU VERGER,1500.0,01007,Maison,7.0,130.0,467.0,5.325067,45.999509,1
40494,2022-13988,2022-11-07,NaN,NaN,CHAMPIONNIERE,1500.0,01007,NaN,NaN,NaN,50.0,5.324937,45.999470,1
54436,2022-18477,2022-03-01,NaN,2.0,RUE JEAN JACQUES ROUSSEAU,2300.0,02173,Maison,4.0,108.0,351.0,3.222222,49.610703,1
54437,2022-18477,2022-03-01,NaN,2.0,RUE JEAN JACQUES ROUSSEAU,2300.0,02173,NaN,0.0,NaN,351.0,3.222222,49.610703,1


In [18]:
# Suppression des lignes avec une valeur Nan dans valeur fonciére
df = df[~df['id_mutation'].isin(liste_id_with_nan_valeur_fonciere)]

In [19]:
# Les Nan dans adresse_numero peuvent correspondre à un lieu dit donc sans numéro, la valeur des Nan sont remplacées par 0.0
df['adresse_numero'] = df['adresse_numero'].fillna("0.0")

In [20]:
# Visualisation des adresse nom de voie avec des nan
liste_id_with_nan_nom_voie = df.loc[pd.isna(df['adresse_nom_voie']), 'id_mutation'].unique()
df_filtered = df[df['id_mutation'].isin(liste_id_with_nan_nom_voie)]
df_filtered.head(10)

,id_mutation,date_mutation,valeur_fonciere,adresse_numero,adresse_nom_voie,code_postal,code_commune,type_local,nombre_pieces_principales,surface_reelle_bati,surface_terrain,longitude,latitude,dependance
38578,2022-13353,2022-10-13,230000.0,0.0,LE VILLAGE,1090.0,01263,NaN,NaN,NaN,425.0,4.757520,46.082578,1
38579,2022-13353,2022-10-13,230000.0,0.0,NaN,NaN,01263,NaN,NaN,NaN,NaN,4.757139,46.082381,1
38580,2022-13353,2022-10-13,230000.0,60.0,RUE DES MINIMES,1090.0,01263,NaN,0.0,NaN,112.0,4.757215,46.082390,1
38581,2022-13353,2022-10-13,230000.0,60.0,RUE DES MINIMES,1090.0,01263,Maison,3.0,85.0,112.0,4.757215,46.082390,1
63149,2022-21369,2022-05-11,105000.0,5.0,NaN,2450.0,02269,NaN,0.0,NaN,120.0,3.676078,49.975309,1
63150,2022-21369,2022-05-11,105000.0,5.0,NaN,2450.0,02269,Maison,2.0,45.0,4960.0,3.676078,49.975309,1
63151,2022-21369,2022-05-11,105000.0,5.0,NaN,2450.0,02269,NaN,0.0,NaN,4960.0,3.676078,49.975309,1
63152,2022-21369,2022-05-11,105000.0,5.0,NaN,2450.0,02269,Maison,2.0,45.0,120.0,3.676078,49.975309,1
63153,2022-21369,2022-05-11,105000.0,5.0,NaN,2450.0,02269,NaN,NaN,NaN,774.0,3.676821,49.975506,1
303564,2022-106151,2022-05-23,425000.0,81.0,RUE BOURBON,8000.0,08105,Maison,6.0,195.0,220.0,4.715592,49.772559,0


In [21]:
# Remplacement des Nan par un texte vide. à noter qu'une même vente peut avoir deux adresse différente Lors du groupement sur l'id on prendra le texte le plus long
df['adresse_nom_voie'] = df['adresse_nom_voie'].fillna("")

In [22]:
# Visualisation des id avec un code postal Nan
liste_id_with_nan_code_postal = df.loc[pd.isna(df['code_postal']), 'id_mutation'].unique()
df_filtered = df[df['id_mutation'].isin(liste_id_with_nan_code_postal)]
df_filtered.head(10)

,id_mutation,date_mutation,valeur_fonciere,adresse_numero,adresse_nom_voie,code_postal,code_commune,type_local,nombre_pieces_principales,surface_reelle_bati,surface_terrain,longitude,latitude,dependance
38578,2022-13353,2022-10-13,230000.0,0.0,LE VILLAGE,1090.0,01263,NaN,NaN,NaN,425.0,4.757520,46.082578,1
38579,2022-13353,2022-10-13,230000.0,0.0,,NaN,01263,NaN,NaN,NaN,NaN,4.757139,46.082381,1
38580,2022-13353,2022-10-13,230000.0,60.0,RUE DES MINIMES,1090.0,01263,NaN,0.0,NaN,112.0,4.757215,46.082390,1
38581,2022-13353,2022-10-13,230000.0,60.0,RUE DES MINIMES,1090.0,01263,Maison,3.0,85.0,112.0,4.757215,46.082390,1
303564,2022-106151,2022-05-23,425000.0,81.0,RUE BOURBON,8000.0,08105,Maison,6.0,195.0,220.0,4.715592,49.772559,0
303565,2022-106151,2022-05-23,425000.0,0.0,,NaN,08105,NaN,NaN,NaN,NaN,4.715624,49.772424,0
380363,2022-129089,2022-03-22,57000.0,0.0,,NaN,11206,NaN,NaN,NaN,NaN,2.215003,43.054207,0
380364,2022-129089,2022-03-22,57000.0,19.0,RUE DE LA FUSTERIE,11300.0,11206,Appartement,1.0,26.0,254.0,2.215055,43.054066,0
380365,2022-129089,2022-03-22,57000.0,19.0,RUE DE LA FUSTERIE,11300.0,11206,Appartement,2.0,32.0,254.0,2.215055,43.054066,0
380366,2022-129089,2022-03-22,57000.0,19.0,RUE DE LA FUSTERIE,11300.0,11206,Appartement,2.0,45.0,254.0,2.215055,43.054066,0


In [23]:
# On remplace le code postal manquant par 0.0 et on prendra le max lors du groupement
df['code_postal'] = df['code_postal'].fillna("0.0")

In [24]:
# Transformation du type de local par "" on prendra le nom le plus long lors du groupement
df['type_local'] = df['type_local'].fillna("")

In [25]:
# Visualisation des nombres de pieces avec un nan
liste_id_with_nan_nb_pieces = df.loc[pd.isna(df['nombre_pieces_principales']), 'id_mutation'].unique()
df_filtered = df[df['id_mutation'].isin(liste_id_with_nan_nb_pieces)]
df_filtered.head(10)

,id_mutation,date_mutation,valeur_fonciere,adresse_numero,adresse_nom_voie,code_postal,code_commune,type_local,nombre_pieces_principales,surface_reelle_bati,surface_terrain,longitude,latitude,dependance
1,2022-2,2022-01-03,143000.0,0.0,CHAMP COCHET,1480.0,01398,,NaN,NaN,84.0,4.847863,46.000587,0
2,2022-2,2022-01-03,143000.0,0.0,CHAMP COCHET,1480.0,01398,,NaN,NaN,88.0,4.847924,46.000581,0
3,2022-2,2022-01-03,143000.0,98.0,RTE DE LA DOMBES,1480.0,01398,Appartement,3.0,140.0,NaN,4.848340,46.000631,0
5,2022-4,2022-01-06,255000.0,282.0,RTE DE POISATON,1560.0,01230,Maison,5.0,108.0,649.0,5.103407,46.422348,0
6,2022-4,2022-01-06,255000.0,0.0,LA FREGONIERE,1560.0,01230,,NaN,NaN,360.0,5.103355,46.422257,0
13,2022-8,2022-01-03,140000.0,6.0,RUE DE BEAUGENCY,1000.0,01053,Maison,4.0,100.0,796.0,5.250556,46.209586,0
14,2022-8,2022-01-03,140000.0,0.0,RUE DE BEAUGENCY,1000.0,01053,,NaN,NaN,260.0,5.249823,46.209593,0
15,2022-8,2022-01-03,140000.0,0.0,RUE DE BEAUGENCY,1000.0,01053,,NaN,NaN,590.0,5.250167,46.209508,0
16,2022-8,2022-01-03,140000.0,0.0,RUE DE BEAUGENCY,1000.0,01053,,NaN,NaN,262.0,5.249705,46.209465,0
20,2022-10,2022-01-10,580000.0,0.0,COUTELIEU,1500.0,01007,,NaN,NaN,27.0,5.343547,45.995066,1


In [26]:
# On va remplacer la valeur des Nan dans nombre_pieces_principales, surface_reelle_bati et surface_terrain par "O.O" 
# Lors du groupement on prendra la valeur max de nombre_pieces_principales et surface_reelle_bati
# Mais on fera la somme de surface_terrain qui correspond à la taille de plusieures parcelles de terrain
df['nombre_pieces_principales'] = df['nombre_pieces_principales'].fillna("0.0")
df['surface_reelle_bati'] = df['surface_reelle_bati'].fillna("0.0")
df['surface_terrain'] = df['surface_terrain'].fillna("0.0")

In [27]:
# Visualisation des id avec des nan dans longitude
liste_id_with_nan_longitude = df.loc[pd.isna(df['longitude']), 'id_mutation'].unique()
df_filtered = df[df['id_mutation'].isin(liste_id_with_nan_longitude)]
df_filtered.head(10)

,id_mutation,date_mutation,valeur_fonciere,adresse_numero,adresse_nom_voie,code_postal,code_commune,type_local,nombre_pieces_principales,surface_reelle_bati,surface_terrain,longitude,latitude,dependance
723,2022-281,2022-01-06,315000.0,526.0,CHE DE FLEYRIAT LE HAUT,1440.0,01451,Maison,7.0,438.0,5920.0,NaN,NaN,0
724,2022-281,2022-01-06,315000.0,526.0,CHE DE FLEYRIAT LE HAUT,1440.0,01451,Maison,7.0,438.0,2169.0,NaN,NaN,0
725,2022-281,2022-01-06,315000.0,526.0,CHE DE FLEYRIAT LE HAUT,1440.0,01451,Maison,7.0,438.0,5664.0,NaN,NaN,0
801,2022-318,2022-01-19,200000.0,184.0,RUE DE L EGLISE,1800.0,01450,Maison,4.0,84.0,80.0,NaN,NaN,0
802,2022-318,2022-01-19,200000.0,0.0,VILLIEU,1800.0,01450,,0.0,0.0,1045.0,NaN,NaN,0
803,2022-318,2022-01-19,200000.0,212.0,RUE DE L EGLISE,1800.0,01450,Maison,3.0,170.0,500.0,NaN,NaN,0
866,2022-347,2022-01-20,124000.0,14.0,BD DE L HIPPODROME,1000.0,01053,,0.0,0.0,0.0,NaN,NaN,1
867,2022-347,2022-01-20,124000.0,14.0,BD DE L HIPPODROME,1000.0,01053,,0.0,0.0,0.0,NaN,NaN,1
868,2022-347,2022-01-20,124000.0,26.0,RUE DE CUIRON,1000.0,01053,,0.0,0.0,0.0,NaN,NaN,1
869,2022-347,2022-01-20,124000.0,14.0,BD DE L HIPPODROME,1000.0,01053,Appartement,4.0,77.0,0.0,NaN,NaN,1


In [28]:
# On supprime les id_mutation avec qui ont une longitude Nan car l'eb demande de faire apparaître les biens sur une carte
df = df.dropna(subset=['longitude'])
df = df.dropna(subset=['latitude'])

In [29]:
### Vérification de la gestion des NaN
df.isna().sum()

id_mutation                  0
date_mutation                0
valeur_fonciere              0
adresse_numero               0
adresse_nom_voie             0
code_postal                  0
code_commune                 0
type_local                   0
nombre_pieces_principales    0
surface_reelle_bati          0
surface_terrain              0
longitude                    0
latitude                     0
dependance                   0
dtype: int64

### Formatage des données avant groupement 

In [30]:
df.head()

,id_mutation,date_mutation,valeur_fonciere,adresse_numero,adresse_nom_voie,code_postal,code_commune,type_local,nombre_pieces_principales,surface_reelle_bati,surface_terrain,longitude,latitude,dependance
0,2022-1,2022-01-03,55000.0,13.0,RUE DE LA LIBERTE,1000.0,01053,Appartement,1.0,24.0,0.0,5.218712,46.198053,0
1,2022-2,2022-01-03,143000.0,0.0,CHAMP COCHET,1480.0,01398,,0.0,0.0,84.0,4.847863,46.000587,0
2,2022-2,2022-01-03,143000.0,0.0,CHAMP COCHET,1480.0,01398,,0.0,0.0,88.0,4.847924,46.000581,0
3,2022-2,2022-01-03,143000.0,98.0,RTE DE LA DOMBES,1480.0,01398,Appartement,3.0,140.0,0.0,4.848340,46.000631,0
5,2022-4,2022-01-06,255000.0,282.0,RTE DE POISATON,1560.0,01230,Maison,5.0,108.0,649.0,5.103407,46.422348,0


In [31]:
df.loc[:, 'date_mutation'] = pd.to_datetime(df['date_mutation'], errors='coerce').dt.date
df.loc[:, 'valeur_fonciere'] = df.valeur_fonciere.astype(int)
df.loc[:, 'adresse_numero'] = df.adresse_numero.astype(float).astype(int)
df.loc[:, 'code_postal'] = df.code_postal.astype(float)
df.loc[:, 'code_postal'] = df.code_postal.astype(int)
df.loc[:, 'nombre_pieces_principales'] = df.nombre_pieces_principales.astype(float).astype(int)
df.loc[:, 'surface_reelle_bati'] = df.surface_reelle_bati.astype(float).astype(int)
df.loc[:, 'surface_terrain'] = df.surface_terrain.astype(float).astype(int)
df.loc[:, 'longitude'] = df.longitude.astype(float)
df.loc[:, 'latitude'] = df.latitude.astype(float)

In [32]:
df.head()

,id_mutation,date_mutation,valeur_fonciere,adresse_numero,adresse_nom_voie,code_postal,code_commune,type_local,nombre_pieces_principales,surface_reelle_bati,surface_terrain,longitude,latitude,dependance
0,2022-1,2022-01-03 00:00:00,55000.0,13,RUE DE LA LIBERTE,1000,01053,Appartement,1,24,0,5.218712,46.198053,0
1,2022-2,2022-01-03 00:00:00,143000.0,0,CHAMP COCHET,1480,01398,,0,0,84,4.847863,46.000587,0
2,2022-2,2022-01-03 00:00:00,143000.0,0,CHAMP COCHET,1480,01398,,0,0,88,4.847924,46.000581,0
3,2022-2,2022-01-03 00:00:00,143000.0,98,RTE DE LA DOMBES,1480,01398,Appartement,3,140,0,4.848340,46.000631,0
5,2022-4,2022-01-06 00:00:00,255000.0,282,RTE DE POISATON,1560,01230,Maison,5,108,649,5.103407,46.422348,0


### Groupement des données sur id_mutation

In [33]:
# Définissez les fonctions d'agrégation pour chaque colonne
aggregation_functions = { 'date_mutation': 'max',   
                          'valeur_fonciere': 'max',
                          'adresse_numero' : 'max',
                          'adresse_nom_voie' : 'max',
                          'code_postal' : 'max',
                          'code_commune' : 'max',
                          'type_local' : 'max',
                          'nombre_pieces_principales' : 'max',
                          'surface_reelle_bati' : 'max',
                          'surface_terrain' : 'sum',
                          'longitude' : 'mean',
                          'latitude' : 'mean',
                          'dependance' : 'max'}  

# Appliquez les fonctions d'agrégation en fonction de la colonne 'Groupe'
df_aggregated = df.groupby('id_mutation').agg(aggregation_functions).reset_index()

In [34]:
df_aggregated.head(10)

,id_mutation,date_mutation,valeur_fonciere,adresse_numero,adresse_nom_voie,code_postal,code_commune,type_local,nombre_pieces_principales,surface_reelle_bati,surface_terrain,longitude,latitude,dependance
0,2022-1,2022-01-03 00:00:00,55000.0,13,RUE DE LA LIBERTE,1000,01053,Appartement,1,24,0,5.218712,46.198053,0
1,2022-10,2022-01-10 00:00:00,580000.0,45,RUE DES COMBES,1500,01007,Maison,6,204,1890,5.343425,45.995108,1
2,2022-10000,2022-07-22 00:00:00,155000.0,54,RUE MARCEL DEMIA,1500,01004,Appartement,3,68,0,5.361497,45.950419,1
3,2022-100000,2022-08-31 00:00:00,143000.0,9003,QUR DE CHEYLARD,7700,07042,Maison,3,80,18679,4.645049,44.384193,0
4,2022-1000003,2022-06-21 00:00:00,93285.0,51,RUE FERNAND TAVERNE,62210,62065,Maison,6,104,1246,2.855865,50.410711,1
5,2022-1000004,2022-05-23 00:00:00,141212.0,20,RUE PEUPLIER,62128,62754,Maison,3,68,419,2.856913,50.182973,0
6,2022-1000007,2022-06-18 00:00:00,159000.0,2,RUE DU DOCTEUR BAUDE,62000,62041,Maison,4,90,110,2.786137,50.283095,0
7,2022-1000008,2022-06-24 00:00:00,25000.0,3,RUE PRINCIPALE,62390,62345,Maison,2,75,1534,2.063507,50.307142,0
8,2022-1000009,2022-06-09 00:00:00,105000.0,22,RUE DU FBG DE PARIS,62159,62839,Maison,2,41,1434,2.908162,50.142687,1
9,2022-1000010,2022-06-30 00:00:00,130000.0,15,RUE DES HAIES,62270,62337,Maison,3,82,3320,2.250969,50.318340,1


In [35]:
print("Nombre de ligne dans le dataframe :",df_aggregated.shape[0])
print("Nombrede vente unique :",df_aggregated.id_mutation.nunique())

Nombre de ligne dans le dataframe : 1035094


Nombrede vente unique : 1035094


---
---
---


<div style="text-align:center; background-color:blue; padding:10px;">
  <h1 style="color:white;">Correlation des données entre le dataframe et la base de données</h1>
</div>


In [36]:
# Pour la vérification on prendra les 5 premières lignes du dataframe
df_correlation = df_aggregated.iloc[:6,:]
df_correlation.head()

,id_mutation,date_mutation,valeur_fonciere,adresse_numero,adresse_nom_voie,code_postal,code_commune,type_local,nombre_pieces_principales,surface_reelle_bati,surface_terrain,longitude,latitude,dependance
0,2022-1,2022-01-03 00:00:00,55000.0,13,RUE DE LA LIBERTE,1000,01053,Appartement,1,24,0,5.218712,46.198053,0
1,2022-10,2022-01-10 00:00:00,580000.0,45,RUE DES COMBES,1500,01007,Maison,6,204,1890,5.343425,45.995108,1
2,2022-10000,2022-07-22 00:00:00,155000.0,54,RUE MARCEL DEMIA,1500,01004,Appartement,3,68,0,5.361497,45.950419,1
3,2022-100000,2022-08-31 00:00:00,143000.0,9003,QUR DE CHEYLARD,7700,07042,Maison,3,80,18679,4.645049,44.384193,0
4,2022-1000003,2022-06-21 00:00:00,93285.0,51,RUE FERNAND TAVERNE,62210,62065,Maison,6,104,1246,2.855865,50.410711,1


### Utilisation de la base de données réalisées dans le notebook Test_scraping_rds_s3

In [7]:
from connection import db,cursor

# Selection de la table datagouv
query="""
USE datagouv;
"""
cursor.execute(query)


def delete_tables() -> None:
    """
    Fonction permettant de supprimer les tables la base de données
    """
    cursor.execute("SET foreign_key_checks = 0")
    cursor.execute("SHOW TABLES;")
    # Parcourir la liste des tables et supprimer chacune d'entre elles
    for table_name in cursor.fetchall():
        drop_table_query = f"DROP TABLE {table_name[0]}"
        cursor.execute(drop_table_query)
        print(f"Table {table_name} supprimée avec succès.")

    # Valider et appliquer les modifications
    db.commit()
    return

def create_database_and_tables(name_db : str, path_sql_file: str) -> None:
    """
    Fonction permettant de créer une base de données et de créer les tables 
    suivant un schéma de base de données
    """
    query=f"""
    CREATE DATABASE IF NOT EXISTS {name_db};
    """
    cursor.execute(query)

    query="""
    USE datagouv;
    """
    cursor.execute(query)

    # Lecture de fichier create_tables
    with open(path_sql_file) as file:
        sql_queries = file.read()
    try :
        for query in sql_queries.split(';'):
            query = query.strip()
            print(query)
            cursor.execute(query)
    except Exception as e:
        print(f"Erreur dans la creation des tables : {e}")
        db.rollback()

    query="""
    SHOW TABLES;
    """
    cursor.execute(query)
    tables = cursor.fetchall()

    # Afficher les tables
    for table in tables:
        print(f"La table {table[0]} à bien été créée")
    return

InterfaceError: (0, '')

In [ ]:
df_correlation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id_mutation                6 non-null      object 
 1   date_mutation              6 non-null      object 
 2   valeur_fonciere            6 non-null      float64
 3   adresse_numero             6 non-null      object 
 4   adresse_nom_voie           6 non-null      object 
 5   code_postal                6 non-null      object 
 6   code_commune               6 non-null      object 
 7   type_local                 6 non-null      object 
 8   nombre_pieces_principales  6 non-null      object 
 9   surface_reelle_bati        6 non-null      object 
 10  surface_terrain            6 non-null      object 
 11  longitude                  6 non-null      float64
 12  latitude                   6 non-null      float64
 13  dependance                 6 non-null      int64  
dty

### Les différences entre le dataframe et le schéma de base de données
- id_mutation n'est plus nécessaire dans le dataframe
- date_mutation n'est pas au format date
- valeur fonciere doit être passé en int 
- adresse_numero RAS
- adresse_nom_voie RAS
- code postal  ajouter un 0 en debut de code postal pour les codes postaux inferieurs à 10000
- code commune RAS
- type local RAS
- nombre_pieces_principales  int
- surface_reelle_bati  à passer en int
- surface_terrain à passer en int
- dependances ajout de cette variable dans la base de données

In [ ]:
df_correlation = df_correlation.iloc[:,1:]

KeyError: 'date_mutation'

In [ ]:
df_correlation.loc[:, 'date_mutation'] = pd.to_datetime(df_correlation['date_mutation'], errors='coerce').dt.date

In [ ]:
df_correlation.loc['valeur_fonciere'] = df_correlation.valeur_fonciere.astype(int)

In [ ]:
df_correlation['code_postal'] = df_correlation['code_postal'].apply(lambda x: x.zfill(5))

In [ ]:
df_correlation.loc['nombre_pieces_principales'] = df_correlation.nombre_pieces_principales.astype(int)
df_correlation.loc['surface_reelle_bati'] = df_correlation.surface_reelle_bati.astype(int)
df_correlation.loc['surface_terrain'] = df_correlation.surface_terrain.astype(int)

<img src="https://raw.githubusercontent.com/rastakoer/certif_app_immo/datagouv_to_rds/datagouv_to_rds/mld.PNG" alt="Schema de la bdd">


In [46]:
from sqlalchemy import create_engine

# # Informations de connexion à la base de données RDS MySQL
# user = '[votre_utilisateur]'
# password = '[votre_mot_de_passe]'
# host = '[votre_host_rds]'
# port = '[votre_port_rds]'
# database = '[votre_nom_de_base_de_donnees]'

# # Créer une connexion à la base de données
# engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')

# # Votre DataFrame
# df = pd.DataFrame({
#     'colonne1': [valeur1, valeur2, ...],
#     'colonne2': [valeur1, valeur2, ...],
#     # Ajoutez d'autres colonnes selon votre DataFrame
# })

# # Envoyer le DataFrame vers la base de données
# df.to_sql(name='nom_de_votre_table', con=engine, if_exists='replace', index=False)

# # Fermer la connexion
# engine.dispose()

start_value
